# Announcing the `hasPattern` Rule feature! 

This allows you to apply regex rule matching to your data's columns with PyDeequ's Verification suite! 

In [5]:
# import pydeequ

import sys
sys.path.append("/Users/calviwan/Desktop/myProjects/python-deequ")
import pydeequ

from pyspark.sql import SparkSession, Row

spark = (SparkSession
    .builder
    .config("spark.jars.packages", pydeequ.deequ_maven_coord)
    .config("spark.jars.excludes", pydeequ.f2j_maven_coord)
    .getOrCreate())


In [9]:
df = spark.sparkContext.parallelize([
            Row(a="foo", creditCard="5130566665286573", email="foo@example.com", ssn="123-45-6789", URL="http://userid@example.com:8080"),
            Row(a="bar", creditCard="4532677117740914", email="bar@example.com", ssn="123456789",   URL="http://foo.com/(something)?after=parens"),
            Row(a="baz", creditCard="3401453245217421", email="foobar@baz.com",  ssn="000-00-0000", URL ="http://userid@example.com:8080")]).toDF()

In [11]:
from pydeequ.checks import *
from pydeequ.verification import *

check = Check(spark, CheckLevel.Error, "Integrity checks")

checkResult = VerificationSuite(spark) \
    .onData(df) \
    .addCheck(
        check.hasPattern(column='email',
                         pattern=r".*@baz.com",
                         assertion=lambda x: x == 1/3) \
        .hasPattern(column='a',
                         pattern=r"ba(r|z)",
                         assertion=lambda x: x == 0/3) \
        .hasPattern(column='email',
                     pattern=r"""(?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*|"(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])""",
                     assertion=lambda x: x == 1.0)) \
    .run()

checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show()

+----------------+-----------+------------+--------------------+-----------------+--------------------+
|           check|check_level|check_status|          constraint|constraint_status|  constraint_message|
+----------------+-----------+------------+--------------------+-----------------+--------------------+
|Integrity checks|      Error|       Error|PatternMatchConst...|          Success|                    |
|Integrity checks|      Error|       Error|PatternMatchConst...|          Failure|Value: 0.66666666...|
|Integrity checks|      Error|       Error|PatternMatchConst...|          Success|                    |
+----------------+-----------+------------+--------------------+-----------------+--------------------+

